<a href="https://colab.research.google.com/github/existinmyhead/Winter22/blob/main/CIS3200/Project3/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyspark

!wget -q 'https://drive.google.com/uc?export=download&id=1wAdLRbT_YgWJ1XRl39JM-lKwJpewdwyM' -O 'Sample.txt'

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('SparkWordCount')
sc = SparkContext.getOrCreate(conf = conf)

sqlContext = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
input_file = sc.textFile('Sample.txt')

input_file.collect()[:10]

['The Project Gutenberg EBook of The Complete Works of William Shakespeare, by',
 'William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **',
 '**     Please follow the copyright guidelines in this file.     **']

In [ ]:
# Split lines by word and remove all characters and empty spaces from text so only letters remain

import re
flatMapVar = input_file.flatMap(lambda line: re.split(r"[ \!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\]\\\^\_\`\{\}\|\~]+", line)).filter(lambda line: len(line) > 0)

flatMapVar.collect()[:20]

['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'The',
 'Complete',
 'Works',
 'of',
 'William',
 'Shakespeare',
 'by',
 'William',
 'Shakespeare',
 'This',
 'eBook',
 'is',
 'for',
 'the',
 'use']

In [ ]:
# Truncate each word to only the first letter, convert case to lowercase, and map

keyPairs = flatMapVar.map(lambda line: (line[:1].lower(), 1))

keyPairs.collect()[:30]

[('t', 1),
 ('p', 1),
 ('g', 1),
 ('e', 1),
 ('o', 1),
 ('t', 1),
 ('c', 1),
 ('w', 1),
 ('o', 1),
 ('w', 1),
 ('s', 1),
 ('b', 1),
 ('w', 1),
 ('s', 1),
 ('t', 1),
 ('e', 1),
 ('i', 1),
 ('f', 1),
 ('t', 1),
 ('u', 1),
 ('o', 1),
 ('a', 1),
 ('a', 1),
 ('a', 1),
 ('n', 1),
 ('c', 1),
 ('a', 1),
 ('w', 1),
 ('a', 1),
 ('n', 1)]

In [ ]:
# Reduce and filter the remaining numbers out

countKeys = keyPairs.reduceByKey(lambda a, b: a + b).filter(lambda key: ord(key[0]) in range(97, 123))

countKeys.collect()[:30]

[('p', 28059),
 ('g', 21167),
 ('c', 34983),
 ('s', 75226),
 ('b', 46001),
 ('i', 62420),
 ('r', 15234),
 ('y', 25926),
 ('l', 32389),
 ('d', 39173),
 ('j', 3372),
 ('h', 61029),
 ('t', 127781),
 ('e', 20409),
 ('o', 43712),
 ('w', 60097),
 ('f', 37186),
 ('u', 9230),
 ('a', 86000),
 ('n', 27313),
 ('m', 56252),
 ('v', 5802),
 ('k', 9535),
 ('q', 2388),
 ('z', 79),
 ('x', 14)]

In [ ]:
countsDF = sqlContext.createDataFrame(countKeys) \
            .withColumnRenamed('_1','Word').withColumnRenamed('_2','Count')

countsDF.toPandas().to_csv('counts.csv', index=False)

countsDF.show()

+----+------+
|Word| Count|
+----+------+
|   p| 28059|
|   g| 21167|
|   c| 34983|
|   s| 75226|
|   b| 46001|
|   i| 62420|
|   r| 15234|
|   y| 25926|
|   l| 32389|
|   d| 39173|
|   j|  3372|
|   h| 61029|
|   t|127781|
|   e| 20409|
|   o| 43712|
|   w| 60097|
|   f| 37186|
|   u|  9230|
|   a| 86000|
|   n| 27313|
+----+------+
only showing top 20 rows



Here's all the above steps merged into one block of code:

In [ ]:
input_file = sc.textFile('Sample.txt')


# Split lines by word and remove all characters and empty spaces from text so only letters remain
import re
flatMapVar = input_file.flatMap(lambda line: re.split(r"[ \!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\]\\\^\_\`\{\}\|\~]+", line)).filter(lambda line: len(line) > 0)


# Truncate each word to only the first letter, convert case to lowercase, and map
keyPairs = flatMapVar.map(lambda line: (line[:1].lower(), 1))


# Reduce and filter the remaining numbers out
countKeys = keyPairs.reduceByKey(lambda a, b: a + b).filter(lambda key: ord(key[0]) in range(97, 123))


# Create dataframe and export csv
countsDF = sqlContext.createDataFrame(countKeys) \
            .withColumnRenamed('_1','Word').withColumnRenamed('_2','Count')

countsDF.toPandas().to_csv('counts.csv', index=False)

countsDF.show()

+----+------+
|Word| Count|
+----+------+
|   p| 28059|
|   g| 21167|
|   c| 34983|
|   s| 75226|
|   b| 46001|
|   i| 62420|
|   r| 15234|
|   y| 25926|
|   l| 32389|
|   d| 39173|
|   j|  3372|
|   h| 61029|
|   t|127781|
|   e| 20409|
|   o| 43712|
|   w| 60097|
|   f| 37186|
|   u|  9230|
|   a| 86000|
|   n| 27313|
+----+------+
only showing top 20 rows

